In [1]:
import pandas as pd
import requests
import json
import re
import sys
import numpy as np
import os
import nltk
import pickle
from sklearn.utils import shuffle
from importlib import reload
from langdetect import detect
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from dateutil.parser import parse
import spacy
from time import time

sys.path.append('../src')

from commons import elastic
from text_processing import text_normalizer
text_normalizer = reload(text_normalizer)
from text_processing import duplicate_finder
from utilities import excel_writer, excel_reader, utils
from text_processing import abbreviations_resolver
abbreviations_resolver = reload(abbreviations_resolver)

stemmer = SnowballStemmer("english")
lmtzr = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')

_abbreviations_resolver = abbreviations_resolver.AbbreviationsResolver([])
_abbreviations_resolver.load_model("../model/abbreviations_dicts")

# Check Lab studies and Geo names

In [2]:
df = excel_reader.ExcelReader().read_df_from_excel("../tmp/combined data.csv")

Read file ../tmp/combined data.csv: 0.01s
Processed file ../tmp/combined data.csv: 0.02s


In [3]:
df["title"] = df["Title"]
df["abstract"] = df["Abstract"]
df["keywords"] = ""
df["identificators"] = ""

In [4]:
df["File"].value_counts()

Lab studies              102
Included (abstract)      100
Non-causal studies       100
High-income countries    100
No intervention          100
Name: File, dtype: int64

In [5]:
from text_processing import search_engine_insensitive_to_spelling
from text_processing import all_column_filler
search_engine_inverted_index = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
        load_abbreviations = True)
search_engine_inverted_index.create_inverted_index(df)

Processed 0 articles
Processed 501 articles
Processed 0 abbreviations
Processed 3000 abbreviations
Processed 6000 abbreviations
Processed 9000 abbreviations
Processed 12000 abbreviations
Processed 15000 abbreviations
Processed 18000 abbreviations
Processed 21000 abbreviations
Processed 24000 abbreviations
Processed 27000 abbreviations
Processed 27129 abbreviations


In [6]:
_all_column_filler = all_column_filler.AllColumnFiller()
df = _all_column_filler.fill_columns_for_df(
        df, search_engine_inverted_index, _abbreviations_resolver, settings_json = {"columns":[
            {"column_filler_class":"GeoNameFinder"},
            {"column_filler_class": "StudyTypeLabeller", "folder": "../model/study_type_multi",
             "meta_folder": "../model/study_type_multi_meta_agg",
             "scibert_model_folder": "../model/scibert_scivocab_uncased"},
        ]})

Started processing  {'column_filler_class': 'GeoNameFinder'}
Read file ../data/provinces.xlsx: 1.53s
Processed file ../data/provinces.xlsx: 1.48s
Read file ../data/districts.xlsx: 1.65s
Processed file ../data/districts.xlsx: 1.58s
Processed 0 items
Processed 501 items
Processed for 60.994033098220825s
Started processing  {'column_filler_class': 'StudyTypeLabeller', 'folder': '../model/study_type_multi', 'meta_folder': '../model/study_type_multi_meta_agg', 'scibert_model_folder': '../model/scibert_scivocab_uncased'}
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
INFO:tensorflow:Using config: {'_model_dir': '../model/study_type_multi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_lo

INFO:tensorflow:Using config: {'_model_dir': '../model/study_type_multi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6e33a00b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done
INFO:tensorflow:Writing example 0 of 502


INFO:tensorflow:Writing example 0 of 502


INFO:tensorflow:*** Example ***


...

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Processed for 47.09088134765625s


In [7]:
excel_writer.ExcelWriter().save_df_in_excel(df, "../tmp/combined_data_processed.xlsx")

Saving...
Saved to ../tmp/combined_data_processed.xlsx


# Lab studies evaluation

In [9]:
lab_study_true_vals = []
lab_study_true_pred = []
for i in range(len(df)):
    if df["File"].values[i] == "Lab studies":
        lab_study_true_vals.append(1)
    else:
        lab_study_true_vals.append(0)
    if "Laboratory study" in df["study_type"].values[i]:
        lab_study_true_pred.append(1)
    else:
        lab_study_true_pred.append(0)

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(lab_study_true_vals, lab_study_true_pred))  
print(classification_report(lab_study_true_vals, lab_study_true_pred))

[[388  12]
 [ 70  32]]
              precision    recall  f1-score   support

           0       0.85      0.97      0.90       400
           1       0.73      0.31      0.44       102

   micro avg       0.84      0.84      0.84       502
   macro avg       0.79      0.64      0.67       502
weighted avg       0.82      0.84      0.81       502



# High income countries evaluation

In [11]:
geo_locations_info_true_vals = []
geo_locations_info_pred = []
for i in range(len(df)):
    if df["File"].values[i] == 'Lab studies':
        continue
    if df["File"].values[i] == 'High-income countries':
        geo_locations_info_true_vals.append(1)
    else:
        geo_locations_info_true_vals.append(0)
    if 'Low-income countries(995$ or less)' in df["world_bankdivision_regions"].values[i] or 'Middle-income countries($996 to $3,895)' in df["world_bankdivision_regions"].values[i]:
        geo_locations_info_pred.append(0)
    elif len(df["world_bankdivision_regions"].values[i]) == 0:
        geo_locations_info_pred.append(0)
    else:
        geo_locations_info_pred.append(1)

In [12]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(geo_locations_info_true_vals, geo_locations_info_pred))  
print(classification_report(geo_locations_info_true_vals, geo_locations_info_pred))

[[192 108]
 [ 24  76]]
              precision    recall  f1-score   support

           0       0.89      0.64      0.74       300
           1       0.41      0.76      0.54       100

   micro avg       0.67      0.67      0.67       400
   macro avg       0.65      0.70      0.64       400
weighted avg       0.77      0.67      0.69       400

